In [1]:
import os,sys,string, time
from array import array
import matplotlib.pyplot as plt
import numpy as np
import uproot
import pandas as pd
from platform import python_version
import scipy
from scipy import stats
import uproot3
import math
from matplotlib.patches import Rectangle
import pickle
import functools
from importlib import reload

import Utilities.Plotter as PT
import Utilities.Constants as Constants
import Utilities.Variables_list as Variables
import Utilities.Functions as Functions
from Utilities.ipython_exit import exit

print ('Success')


Success


# Setting which samples to pickle

In [2]:
Params = {"Run":"run2a", #The run number, so far either "run1" or "run3"
          "Load_lepton_signal":False, #Just e+e- samples, Majorana
          "Load_pi0_signal":False, #Just pi0 samples, Majorana
          "Load_lepton_dirac":False, #Just e+e-, Dirac
          "Load_pi0_dirac":False, #Just pi0, Dirac
          "Load_standard_bkgs":True, #Backgrounds, 'overlay', 'dirtoverlay' and 'beamoff'
          "Load_data":True, #'beamgood' sample
          "Load_DetVars":False, #overlay Detector variations
          "Load_Signal_DetVars":False, #e+e- Detector variation samples
          "Load_pi0_signal_DetVars":False, #pi0 Detector variation samples
          "Only_keep_common_DetVar_evs":True, #For DetVar samples, only keep events common between samples
          "FLATTEN":True, #Have one row per reconstructed object in the dataframe
          "only_presel":False, #Create small files containing only variables necessary for pre-selection, for making pre-selection plots
          "Load_truth_vars":False, #Load the "mc_" type variables for signal
          "Load_single_file":False, #This will override everything else, put the desired file in the "single_file" line
          "single_file":'beamgood'} 

Params, samples = Functions.new_create_sample_list(Params)
sample_locs_new = Functions.Get_all_sample_locs(Params)

print("Saving these variables: \n" + str(Params["variables_MC"]))


Loading these run2a samples: 
['overlay', 'dirtoverlay', 'beamoff', 'beamgood']
Saving these variables: 
['run', 'sub', 'evt', 'nslice', 'n_pfps', 'n_tracks', 'n_showers', 'swtrig_pre', 'swtrig_post', 'trk_sce_start_x_v', 'trk_sce_start_y_v', 'trk_sce_start_z_v', 'trk_sce_end_x_v', 'trk_sce_end_y_v', 'trk_sce_end_z_v', 'shr_theta_v', 'shr_phi_v', 'shr_px_v', 'shr_py_v', 'shr_pz_v', 'shrclusdir0', 'shrclusdir1', 'shrclusdir2', 'shr_energy_tot', 'trk_theta_v', 'trk_phi_v', 'trk_dir_x_v', 'trk_dir_y_v', 'trk_dir_z_v', 'trk_energy', 'trk_energy_hits_tot', 'trk_energy_tot', 'trk_score_v', 'trk_calo_energy_u_v', 'trk_end_x_v', 'trk_chipr_best', 'pfnplanehits_U', 'pfnplanehits_V', 'pfnplanehits_Y', 'NeutrinoEnergy2', 'SliceCaloEnergy2', 'nu_flashmatch_score', 'contained_sps_ratio', 'flash_time', 'contained_fraction', 'trk_score', 'crtveto', 'shr_tkfit_dedx_U', 'shr_tkfit_dedx_V', 'shr_tkfit_dedx_Y', 'shr_tkfit_dedx_max', 'shr_tkfit_2cm_dedx_Y', 'shr_chipr', 'trk_bragg_p', 'trk_bragg_p_v', 'tr

## Loading and pickling files

In [ ]:
common_evs = Functions.Make_common_evs(samples, sample_locs_new, Params)
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

Functions.New_load_and_pkl(samples, sample_locs_new, loc_pkls, common_evs, Params, save_str="_full_Finished")

## Getting runs in goodrun list

In [3]:
def Get_runs_in_goodruns(root_file_path, goodruns_list):
    """
    Input .root file and \"goodrun\" list.
    Return lists of runs in list and runs not in list.
    """
    root_file = uproot3.open(root_file_path)['nuselection/NeutrinoSelectionFilter']
    df_file = root_file.pandas.df(['run'], flatten=False)
    
    with open(goodruns_list, "r") as file:
        goodruns = [int(line.strip()) for line in file]
        
    unique_values = list(set(goodruns))
    unique_file_subruns = list(set(np.array(df_file['run'])))
    
    runs_in_goodruns, runs_not_in_goodruns = [], []
    
    for run in unique_file_subruns:
        if run in unique_values: runs_in_goodruns.append(run)
        if run not in unique_values: runs_not_in_goodruns.append(run)
        
    return runs_in_goodruns, runs_not_in_goodruns

def Filter_df_runs(df, goodruns_list):
    """
    Input dataframe and goodruns list.
    Return the filtered dataframe, with only the goodruns included.
    """
    filtered = df.loc[(df['run'].isin(goodruns_list))]
    
    return filtered
    
def Return_runs_subruns(df):
    """
    Input filtered df.
    Return lists of runs and subruns.
    """
    # if pd.Series(['run', 'sub']).isin(df.columns).all():
    #     run_sub_list = []
    #     for entry in df.index: #Looping over all events in the dataframe
    #         run_sub = str(df['run'][entry]) + "_" + str(df['sub'][entry])
    #         run_sub_list.append(run_sub)
    #     unique_run_subrun = list(set(run_sub_list))
    #     return df.copy()
    # else:
    #     print("Dataframe needs \"run\", \"sub\" columns.")
    #     return 0
    placeholder=df.drop_duplicates(subset=["run","sub"]).copy()
    runs_list = placeholder["run"]
    subruns_list = placeholder["sub"]
    
    return runs_list, subruns_list

def Save_runs_subruns_as_txt(runs_list, subruns_list, savename):
    """
    Input list of runs and subruns.
    Saves runs subruns file as savename.
    """
    with open(savename, "w") as file:
        for val1, val2 in zip(runs_list, subruns_list):
            file.write(str(val1) + " " + str(val2) + "\n")
    

In [4]:
runs_in_goodruns_2a, not_in_goodruns_2a = Get_runs_in_goodruns("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root", "../NuMI_data/run2_goodruns.txt")
runs_in_goodruns_2b, not_in_goodruns_2b = Get_runs_in_goodruns("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root", "../NuMI_data/run2_goodruns.txt")

print(len(runs_in_goodruns_2a), len(not_in_goodruns_2a))
print(len(runs_in_goodruns_2b), len(not_in_goodruns_2b))

print(type(runs_in_goodruns_2a))

1277 454
899 122
<class 'list'>


In [13]:
r2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamon.root")['nuselection/NeutrinoSelectionFilter']
full_run2a_df = r2a_file.pandas.df(['run', 'sub'], flatten=False)
r2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamon.root")['nuselection/NeutrinoSelectionFilter']
full_run2b_df = r2b_file.pandas.df(['run', 'sub'], flatten=False)

print("run2a beamon length: " + str(len(full_run2a_df)))
print("run2b beamon length: " + str(len(full_run2b_df)))

all_run2a_runs = list(set(np.array(full_run2a_df['run'])))
all_run2b_runs = list(set(np.array(full_run2b_df['run'])))

print("run2a beamon all runs: " + str(len(all_run2a_runs)))
print("run2b beamon all runs: " + str(len(all_run2b_runs)))

run2a_filtered = Filter_df_runs(full_run2a_df, all_run2a_runs)
run2b_filtered = Filter_df_runs(full_run2b_df, all_run2b_runs)

print("run2a beamon length: " + str(len(run2a_filtered)))
print("run2b beamon length: " + str(len(run2b_filtered)))

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print("run2a")
print(len(run2a_runs), len(run2a_subruns))
print("run2b")
print(len(run2b_runs), len(run2b_subruns))

run2a beamon length: 762814
run2b beamon length: 283307
run2a beamon all runs: 1186
run2b beamon all runs: 479
run2a beamon length: 762814
run2b beamon length: 283307
run2a
152856 152856
run2b
54228 54228


In [14]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/FULL_beamon_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/FULL_beamon_run2b_run_sub.txt")

In [16]:
r2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root")['nuselection/NeutrinoSelectionFilter']
full_run2a_df = r2a_file.pandas.df(['run', 'sub'], flatten=False)
r2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root")['nuselection/NeutrinoSelectionFilter']
full_run2b_df = r2b_file.pandas.df(['run', 'sub'], flatten=False)

print("run2a beamon length: " + str(len(full_run2a_df)))
print("run2b beamon length: " + str(len(full_run2b_df)))

all_run2a_runs = list(set(np.array(full_run2a_df['run'])))
all_run2b_runs = list(set(np.array(full_run2b_df['run'])))

print("run2a beamoff all runs: " + str(len(all_run2a_runs)))
print("run2b beamoff all runs: " + str(len(all_run2b_runs)))

run2a_filtered = Filter_df_runs(full_run2a_df, all_run2a_runs)
run2b_filtered = Filter_df_runs(full_run2b_df, all_run2b_runs)

print("run2a beamon length: " + str(len(run2a_filtered)))
print("run2b beamon length: " + str(len(run2b_filtered)))

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print("run2a")
print(len(run2a_runs), len(run2a_subruns))
print("run2b")
print(len(run2b_runs), len(run2b_subruns))

run2a beamon length: 1641015
run2b beamon length: 1044584
run2a beamoff all runs: 1731
run2b beamoff all runs: 1021
run2a beamon length: 1641015
run2b beamon length: 1044584
run2a
215845 215845
run2b
109500 109500


In [17]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/FULL_beamoff_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/FULL_beamoff_run2b_run_sub.txt")

In [32]:
run2a_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2a_beamoff.root")['nuselection/NeutrinoSelectionFilter']
run2b_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamoff.root")['nuselection/NeutrinoSelectionFilter']

run2a_df = run2a_file.pandas.df(['run', 'sub', 'evt'], flatten=False)
run2b_df = run2b_file.pandas.df(['run', 'sub', 'evt'], flatten=False)

run2a_filtered = Filter_df_runs(run2a_df, runs_in_goodruns_2a)
run2b_filtered = Filter_df_runs(run2b_df, runs_in_goodruns_2b)

run2a_runs, run2a_subruns = Return_runs_subruns(run2a_filtered)
run2b_runs, run2b_subruns = Return_runs_subruns(run2b_filtered)

print(len(run2a_runs), len(run2a_subruns))
print(len(run2b_runs), len(run2b_subruns))


185419 185419
104922 104922


In [33]:
Save_runs_subruns_as_txt(run2a_runs, run2a_subruns, "../NuMI_data/beamoff_run2a_run_sub.txt")
Save_runs_subruns_as_txt(run2b_runs, run2b_subruns, "../NuMI_data/beamoff_run2b_run_sub.txt")

In [6]:
file_locs = Functions.Get_all_sample_locs(Params)

In [7]:
root_file_loc = file_locs["beamgood"]
print(root_file_loc)

../NuMI_data/neutrinoselection_filt_run2a_beamon.root


In [8]:
file_path = "../NuMI_data/run2_goodruns.txt"  

with open(file_path, "r") as file:
    values = [int(line.strip()) for line in file]
    
root_file = uproot3.open(root_file_loc)['nuselection/NeutrinoSelectionFilter']

df_file = root_file.pandas.df(['run', 'sub', 'evt'], flatten=False)


In [30]:
unique_values = list(set(values))
unique_file_subruns = list(set(np.array(df_file['run'])))

runs_in_goodruns = []
runs_not_in_goodruns = []

for run in unique_file_subruns:
    if run in unique_values: runs_in_goodruns.append(run)
    if run not in unique_values: runs_not_in_goodruns.append(run)
    
print("Total runs in run2a file is " + str(len(unique_file_subruns)))
print("runs in goodrun list: " + str(len(runs_in_goodruns)))
print("runs NOT in goodrun list: " + str(len(runs_not_in_goodruns)))

Total runs in run2a file is 479
runs in goodrun list: 453
runs NOT in goodrun list: 26


In [22]:
file_path = "../NuMI_data/run2_goodruns.txt"  

with open(file_path, "r") as file:
    values = [int(line.strip()) for line in file]
    
root_file = uproot3.open("../NuMI_data/neutrinoselection_filt_run2b_beamon.root")['nuselection/NeutrinoSelectionFilter']

df_file = root_file.pandas.df(['run', 'sub', 'evt'], flatten=False)

In [23]:
unique_values = list(set(values))
unique_file_subruns = list(set(np.array(df_file['run'])))

runs_in_goodruns = []
runs_not_in_goodruns = []

for run in unique_file_subruns:
    if run in unique_values: runs_in_goodruns.append(run)
    if run not in unique_values: runs_not_in_goodruns.append(run)
    
print("Total runs in run2b file is ")
print("runs in goodrun list: " + str(len(runs_in_goodruns)))
print("runs NOT in goodrun list: " + str(len(runs_not_in_goodruns)))

Total runs in file is 
runs in goodrun list: 453
runs NOT in goodrun list: 26


## End of code

In [ ]:
print(Params["variables_MC"])

In [ ]:
root_dir = 'nuselection'
main_tree = 'NeutrinoSelectionFilter'
loc_pkls = "pkl_files/"+Params["Run"]+"/current_files/"+Params["variables_string"]+"/"

if (Params["Load_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True):
    df_rse_list = []
    print("Loading only run, subrun, event numbers" + "\n")
    for sample in samples: #Looping over all samples, should make a function for this.
        if sample in Constants.Detector_variations: #Checks if it is an overlay Detector Variation sample
            print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_overlay = uproot3.open("../NuMI_MC/DetVars/neutrinoselection_filt_"+Params["Run"]+f"_overlay_{sample}.root")[root_dir+"/"+main_tree]
            df_overlay = NuMI_MC_overlay.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_overlay
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_overlay = file.copy()
            df_rse_list.append(new_overlay)

    common_evs = Functions.make_common_evs_df(df_rse_list)
    
elif (Params["Load_Signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = [150]
    if Params["Run"] == "run3": HNL_samples = [2, 10, 20, 50, 100]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
        
elif (Params["Load_pi0_signal_DetVars"] == True) and (Params["Only_keep_common_DetVar_evs"] == True): #WRONG
    common_evs_dict = {}
    print("Loading only run, subrun, event numbers" + "\n")
    if Params["Run"] == "run1": HNL_samples = []
    if Params["Run"] == "run3": HNL_samples = [150,180,200,220,240,245]
    for HNL_mass in HNL_samples: #Currently don't have 150MeV detvar samples for run3
        print(HNL_mass)
        df_rse_list = []
        for DetVar in Constants.Detector_variations: #Looping over all samples, should make a function for this.
            #print(f"Loading overlay {sample} "+Params["Run"]+" with uproot")
            NuMI_MC_signal = uproot3.open("../NuMI_signal/KDAR_dump/sfnues/pi0/DetVars/"+f"{HNL_mass}_{DetVar}_"+Params["Run"]+".root")[root_dir+"/"+main_tree]
            df_signal = NuMI_MC_signal.pandas.df(['run','sub','evt'], flatten=Params["FLATTEN"])
            file = df_signal
            Functions.make_unique_ev_id(file) #This creates "rse_id" branch
            new_signal = file.copy()
            df_rse_list.append(new_signal)

        common_evs = Functions.make_common_evs_df(df_rse_list)
        common_evs_dict[HNL_mass] = common_evs
else: common_evs = None

## Making new variables

In [ ]:
def Make_first_new_variables(df): #Only for variables which can be directly made from the ntuple variables with no manipulation/selection
    df['subcluster'] = df['shrsubclusters0'] + df['shrsubclusters1'] + df['shrsubclusters2']
    df['trkfit'] = df['shr_tkfit_npointsvalid'] / df['shr_tkfit_npoints']
    #Something like shr_tkfit_distance
    return df.copy

#Works, but takes a while, ignored objects with huge negative numbers for the positions.
def Make_fiducial_vars(df):
    print(len(df))
    n_pfps = df["n_pfps"].groupby(level="entry").apply(max)
    df_placeholder = df.query("trk_sce_start_x_v>-1e4").copy()
    print(len(df_placeholder))
    min_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_x=df_placeholder[["trk_sce_start_x_v","trk_sce_end_x_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_y=df_placeholder[["trk_sce_start_y_v","trk_sce_end_y_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    min_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].min(axis=1).groupby(level="entry").apply(min)
    max_z=df_placeholder[["trk_sce_start_z_v","trk_sce_end_z_v"]].max(axis=1).groupby(level="entry").apply(max)
    
    del df_placeholder
    
    df2 = df.copy()
    
    df2["min_x"]=np.array(np.repeat(min_x, np.array(n_pfps)))
    df2["max_x"]=np.array(np.repeat(max_x, np.array(n_pfps)))
    df2["min_y"]=np.array(np.repeat(min_y, np.array(n_pfps)))
    df2["max_y"]=np.array(np.repeat(max_y, np.array(n_pfps)))
    df2["min_z"]=np.array(np.repeat(min_z, np.array(n_pfps)))
    df2["max_z"]=np.array(np.repeat(max_z, np.array(n_pfps)))
    return df2

# Reading in and pickling files

In [ ]:
if ((Params["Load_Signal_DetVars"] == True)or(Params["Load_pi0_signal_DetVars"] == True)) and (Params["Only_keep_common_DetVar_evs"] == True):
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs_dict, Params, save_str="_FINAL")
else:
    Functions.Load_and_pkl_samples(samples, sample_loc, loc_pkls, common_evs, Params, save_str="_FINAL") #Remove "_New_gen" str for data
print("Done!")

# New_load_and_pkl(samples, sample_loc, loc_pkls, common_evs, Params, save_str="")

# Finished code